### ФИО: Бекетов Роман

In [1]:
# Install required packages.
'%%capture'

!pip install torch-geometric
!pip install sentence_transformers

  Obtaining dependency information for torch-geometric from https://files.pythonhosted.org/packages/65/4e/6f9a75548a93fedcd4514ae2de9bee1e91bade6b73252b4da32f0e42ac52/torch_geometric-2.4.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=f14f16432e96fb2d6eab42694710c8294b1eb8b237276893d6590e39d7fccb28
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [18]:
import torch
import torch.nn.functional as F
from torch.nn import Linear
from torch_geometric.datasets import MovieLens
from torch_geometric.nn import GCNConv, SAGEConv, to_hetero
import torch_geometric.transforms as T
from torch_geometric.nn import GATConv

from tqdm import tqdm
import sys
import numpy as np

In [3]:
torch.manual_seed(42)

if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

np.random.seed(42)

In [4]:
dataset_path = '/tmp/'
dataset = MovieLens(root=dataset_path)



Extracting /tmp/raw/ml-latest-small.zip
Processing...


.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

Batches:   0%|          | 0/305 [00:00<?, ?it/s]

Done!


In [5]:
dataset[0]

HeteroData(
  movie={ x=[9742, 404] },
  user={ num_nodes=610 },
  (user, rates, movie)={
    edge_index=[2, 100836],
    edge_label=[100836],
  }
)

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

data = dataset[0].to(device)

# Add user node features for message passing:
data['user'].x = torch.eye(data['user'].num_nodes, device=device)
del data['user'].num_nodes

# Add a reverse ('movie', 'rev_rates', 'user') relation for message passing:
data = T.ToUndirected()(data)
del data['movie', 'rev_rates', 'user'].edge_label  # Remove "reverse" label.

# Perform a link-level split into training, validation, and test edges:
train_data, val_data, test_data = T.RandomLinkSplit(
    num_val=0.1,
    num_test=0.1,
    neg_sampling_ratio=0.0,
    edge_types=[('user', 'rates', 'movie')],
    rev_edge_types=[('movie', 'rev_rates', 'user')],
)(data)


weight = torch.bincount(train_data['user', 'movie'].edge_label)
weight = weight.max() / weight


def weighted_mse_loss(pred, target, weight=None):
    weight = 1. if weight is None else weight[target].to(pred.dtype)
    return (weight * (pred - target.to(pred.dtype)).pow(2)).mean()


class GNNEncoder(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv((-1, -1), hidden_channels)
        self.conv2 = SAGEConv((-1, -1), out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x


class EdgeDecoder(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.lin1 = Linear(2 * hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, 1)

    def forward(self, z_dict, edge_label_index):
        row, col = edge_label_index
        z = torch.cat([z_dict['user'][row], z_dict['movie'][col]], dim=-1)

        z = self.lin1(z).relu()
        z = self.lin2(z)
        return z.view(-1)


class Model(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.encoder = GNNEncoder(hidden_channels, hidden_channels)
        self.encoder = to_hetero(self.encoder, data.metadata(), aggr='sum')
        self.decoder = EdgeDecoder(hidden_channels)

    def forward(self, x_dict, edge_index_dict, edge_label_index):
        z_dict = self.encoder(x_dict, edge_index_dict)
        return self.decoder(z_dict, edge_label_index)


def train():
    model.train()
    optimizer.zero_grad()
    pred = model(train_data.x_dict, train_data.edge_index_dict,
                 train_data['user', 'movie'].edge_label_index)
    target = train_data['user', 'movie'].edge_label
    loss = weighted_mse_loss(pred, target, weight)
    loss.backward()
    optimizer.step()
    return float(loss)


@torch.no_grad()
def test(data):
    model.eval()
    pred = model(data.x_dict, data.edge_index_dict,
                 data['user', 'movie'].edge_label_index)
    pred = pred.clamp(min=0, max=5)
    target = data['user', 'movie'].edge_label.float()
    rmse = F.mse_loss(pred, target).sqrt()
    return float(rmse)




In [7]:
train_data, test_data

(HeteroData(
   movie={ x=[9742, 404] },
   user={ x=[610, 610] },
   (user, rates, movie)={
     edge_index=[2, 80670],
     edge_label=[80670],
     edge_label_index=[2, 80670],
   },
   (movie, rev_rates, user)={ edge_index=[2, 80670] }
 ),
 HeteroData(
   movie={ x=[9742, 404] },
   user={ x=[610, 610] },
   (user, rates, movie)={
     edge_index=[2, 90753],
     edge_label=[10083],
     edge_label_index=[2, 10083],
   },
   (movie, rev_rates, user)={ edge_index=[2, 90753] }
 ))

In [15]:
model = Model(hidden_channels=128).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=2e-3)

## Задание

1) Подберите оптимальные параметры для сети из примера выше(2 балла)

2) Попробуйте вместо GraphSage модуль Graph Attention и также подберите оптимальные параметры  (2 балла)


***По последнему заданию давайте считать, что задача найти архитектуру, для которой RMSE на тесте лучше 1.1*** *(c) Кузьма Храбров 12 декабря 19:10*

In [16]:
sys.stdout.flush()
for epoch in tqdm(range(1, 300), desc='Training Progress', position=0, leave=True):
    loss = train()
    train_rmse = test(train_data)
    val_rmse = test(val_data)
    test_rmse = test(test_data)
    print(f'\rEpoch: {epoch:03d}, Loss: {loss:.4f}, Train: {train_rmse:.4f}, '
          f'Val: {val_rmse:.4f}, Test: {test_rmse:.4f}')

Training Progress:   1%|▏         | 4/299 [00:00<00:16, 18.40it/s]

Epoch: 001, Loss: 19.5845, Train: 3.4182, Val: 3.4082, Test: 3.4026
Epoch: 002, Loss: 18.2158, Train: 3.2797, Val: 3.2701, Test: 3.2648
Epoch: 003, Loss: 16.7818, Train: 3.0613, Val: 3.0529, Test: 3.0479
Epoch: 004, Loss: 14.6913, Train: 2.7120, Val: 2.7056, Test: 2.7014


Training Progress:   3%|▎         | 8/299 [00:00<00:15, 18.76it/s]

Epoch: 005, Loss: 11.7849, Train: 2.1827, Val: 2.1794, Test: 2.1768
Epoch: 006, Loss: 8.4372, Train: 1.4710, Val: 1.4709, Test: 1.4732
Epoch: 007, Loss: 6.1858, Train: 1.1081, Val: 1.1011, Test: 1.1150
Epoch: 008, Loss: 8.3386, Train: 1.1597, Val: 1.1504, Test: 1.1653


Training Progress:   4%|▍         | 12/299 [00:00<00:15, 18.96it/s]

...

Epoch: 289, Loss: 2.1189, Train: 0.9988, Val: 1.1086, Test: 1.1103
Epoch: 290, Loss: 2.1210, Train: 0.9606, Val: 1.0708, Test: 1.0753
Epoch: 291, Loss: 2.1237, Train: 1.0078, Val: 1.1178, Test: 1.1191
Epoch: 292, Loss: 2.1248, Train: 0.9566, Val: 1.0670, Test: 1.0719


Training Progress:  99%|█████████▉| 296/299 [00:15<00:00, 19.13it/s]

Epoch: 293, Loss: 2.1247, Train: 1.0067, Val: 1.1168, Test: 1.1183
Epoch: 294, Loss: 2.1208, Train: 0.9599, Val: 1.0704, Test: 1.0751
Epoch: 295, Loss: 2.1154, Train: 0.9953, Val: 1.1058, Test: 1.1082
Epoch: 296, Loss: 2.1087, Train: 0.9703, Val: 1.0812, Test: 1.0852


Training Progress: 100%|██████████| 299/299 [00:15<00:00, 19.07it/s]

Epoch: 297, Loss: 2.1034, Train: 0.9813, Val: 1.0925, Test: 1.0957
Epoch: 298, Loss: 2.0998, Train: 0.9826, Val: 1.0941, Test: 1.0973
Epoch: 299, Loss: 2.0982, Train: 0.9692, Val: 1.0810, Test: 1.0852


## TEST RMSE: 1.0852

## **Graph Attention**

In [29]:
class GATNEncoder(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GATConv((-1, -1), hidden_channels, heads=1, add_self_loops=False)
        self.conv2 = GATConv((-1, -1), out_channels, heads=1, add_self_loops=False)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x


class EdgeDecoder(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.lin1 = Linear(2 * hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, 1)

    def forward(self, z_dict, edge_label_index):
        row, col = edge_label_index
        z = torch.cat([z_dict['user'][row], z_dict['movie'][col]], dim=-1)

        z = self.lin1(z).relu()
        z = self.lin2(z)
        return z.view(-1)


class Model(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.encoder = GATNEncoder(2*hidden_channels, hidden_channels)
        self.encoder = to_hetero(self.encoder, data.metadata(), aggr='sum')
        self.decoder = EdgeDecoder(hidden_channels)

    def forward(self, x_dict, edge_index_dict, edge_label_index):
        z_dict = self.encoder(x_dict, edge_index_dict)
        return self.decoder(z_dict, edge_label_index)

In [46]:
model = Model(hidden_channels=128).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=2e-3)

In [47]:
for epoch in tqdm(range(1, 500), desc='Training Progress', position=0, leave=True):
    loss = train()
    train_rmse = test(train_data)
    val_rmse = test(val_data)
    test_rmse = test(test_data)
    sys.stdout.flush()
    print(f'\rEpoch: {epoch:03d}, Loss: {loss:.4f}, Train: {train_rmse:.4f}, '
          f'Val: {val_rmse:.4f}, Test: {test_rmse:.4f}')
    if test_rmse < 1.1:
        break
    

Training Progress:   0%|          | 0/499 [00:00<?, ?it/s]

Epoch: 001, Loss: 19.9851, Train: 3.4768, Val: 3.4683, Test: 3.4628


Training Progress:   0%|          | 2/499 [00:00<00:26, 18.47it/s]

Epoch: 002, Loss: 18.8480, Train: 3.3471, Val: 3.3427, Test: 3.3375
Epoch: 003, Loss: 17.4702, Train: 3.1514, Val: 3.1521, Test: 3.1471


Training Progress:   1%|          | 4/499 [00:00<00:27, 18.26it/s]

...

Epoch: 268, Loss: 2.6526, Train: 1.1021, Val: 1.1688, Test: 1.1682
Epoch: 269, Loss: 2.6629, Train: 1.0331, Val: 1.1014, Test: 1.1041


Training Progress:  54%|█████▍    | 270/499 [00:14<00:12, 18.52it/s]

Epoch: 270, Loss: 2.6819, Train: 1.1379, Val: 1.2030, Test: 1.2011


Training Progress:  54%|█████▍    | 270/499 [00:14<00:12, 18.33it/s]

Epoch: 271, Loss: 2.7168, Train: 1.0189, Val: 1.0873, Test: 1.0912


## TEST RMSE: 1.0912